In [11]:
import pandas as pd
import numpy as np

import logging
import ast

from sentence_transformers import SentenceTransformer
from chromadb.utils.embedding_functions import SentenceTransformerEmbeddingFunction


In [12]:
pd.set_option('display.max_columns', 100)
pd.set_option('display.width', 1000)
pd.set_option('max_colwidth', 100)

In [13]:
# Configure logging
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s'
)
logger = logging.getLogger(__name__)

In [14]:
EMBEDDING_MODEL =  'paraphrase-multilingual-mpnet-base-v2'

In [15]:
def preprocess_product_data(df: pd.DataFrame) -> pd.DataFrame:
    """
    Preprocess product dataset
    """
    df.fillna('', inplace=True)
    df['product_gender'] = df['product_gender'].apply(ast.literal_eval).apply(lambda x: ','.join(x)) 
    df['product_style'] = df['product_style'].apply(ast.literal_eval).apply(lambda x: ', '.join(x)) 
    df['product_note'] = df['product_note'].apply(ast.literal_eval).apply(lambda x: ', '.join(x)) 
    df['combined_text'] = (
        df['full_name'].str.strip() + ' ' +
        df['product_gender'].str.strip() + ' ' +
        df['brand'].str.strip() + ' ' +
        df['description'].str.strip() + ' ' +
        df['origin'].str.strip() + ' ' +
        df['product_style'].str.strip() + ' ' +
        df['product_note'].str.strip()
    )

    numeric_cols = df.drop(['parent_id'], axis=1).dtypes[(df.dtypes=='int64') | (df.dtypes=='float64')].index.values
    for col in numeric_cols:
        df[col] = pd.to_numeric(df[col], errors='coerce')

    df = df[df['price'] >= 0]
    df = df[df['product_gender'] != '']
    df = df[df['description']!='']    
    return df

In [16]:
logger.info("Loading datasets...")
df = pd.read_csv('../data/namperfume_product.csv')
processed = preprocess_product_data(df)
print(processed.shape)
processed.head(1)

2025-03-21 11:50:57,166 - INFO - Loading datasets...


(1538, 25)


,id,name,full_name,parent_handle,sku,production_year,origin,price,compare_at_price,parent_id,total_quantity,sold_quantity,brand,product_gender,product_style,product_note,rate,count_rate,count_rate_1,count_rate_2,count_rate_3,count_rate_4,count_rate_5,description,combined_text
0,1003772321,Dsquared2 Icon Pour Homme,Dsquared2 Icon Pour Homme - 100ml Tester,dsquared2-icon-pour-homme,110100204700,2024,Ý,2200000,0,1003772317,0,0,DSQUARED2,Nam,"Tinh tế, Nam tính, Cuốn hút","Hương gừng, Cây xô thơm",0.0,0,0,0,0,0,0,"Hương đầu: Gừng, Cam chanh Hương giữa: Hoa tulip, Xô thơm, Phong lữ, Oải hương Hương cuối: Akiga...","Dsquared2 Icon Pour Homme - 100ml Tester Nam DSQUARED2 Hương đầu: Gừng, Cam chanh Hương giữa: Ho..."


In [17]:
def create_product_embeddings(df: pd.DataFrame, model_name: str = EMBEDDING_MODEL) -> np.ndarray:
    """
    Create embeddings for product descriptions
    """
    logger.info(f"Creating embeddings using {model_name}...")
    
    model = SentenceTransformer(model_name)
    texts = df.apply(
        lambda x: f"{x['combined_text']}", 
        axis=1
    ).tolist()
    embeddings = model.encode(texts,show_progress_bar=True,batch_size=32)
    
    return embeddings

In [19]:
import chromadb

chroma_client = chromadb.PersistentClient(path="chroma_db_v1")
collection = chroma_client.get_or_create_collection(name="new_documents")

processed_list = processed.to_dict(orient="records")
embed = create_product_embeddings(processed).tolist()
for i, product in enumerate(processed_list):
    collection.add(
        ids=[str(product['id'])],  
        embeddings=[embed[i]],
        metadatas=[{
            "name": product["name"],
            "description": product["description"],
            "product_style": product['product_style'],
            "production_year": product["production_year"],
            'origin': product['origin'], 
            'price': product['price'], 
            'compare_at_price': product['compare_at_price'], 
            'sold_quantity': product['sold_quantity'],
            'brand': product['brand'],
            'product_gender': product['product_gender'], 
            'product_note': product['product_note'], 
            'rate': product['rate'], 
            'count_rate': product['count_rate'], 
            'count_rate_1': product['count_rate_1'], 
            'count_rate_2': product['count_rate_2'], 
            'count_rate_3': product['count_rate_3'],
            'count_rate_4': product['count_rate_4'],
            'count_rate_5': product['count_rate_5']
            
            }]
    )

print(f"Stored {collection.count()} documents in ChromaDB.")


2025-03-21 12:03:26,554 - INFO - Creating embeddings using paraphrase-multilingual-mpnet-base-v2...
2025-03-21 12:03:26,557 - INFO - Use pytorch device_name: mps
2025-03-21 12:03:26,557 - INFO - Load pretrained SentenceTransformer: paraphrase-multilingual-mpnet-base-v2
Batches: 100%|██████████| 49/49 [02:52<00:00,  3.53s/it]
2025-03-21 12:07:43,462 - WARNING - Add of existing embedding ID: 1003772321
2025-03-21 12:07:43,463 - WARNING - Insert of existing embedding ID: 1003772321
2025-03-21 12:07:43,469 - WARNING - Add of existing embedding ID: 1003772317
2025-03-21 12:07:43,470 - WARNING - Insert of existing embedding ID: 1003772317
2025-03-21 12:07:43,472 - WARNING - Add of existing embedding ID: 1003772262
2025-03-21 12:07:43,472 - WARNING - Insert of existing embedding ID: 1003772262
2025-03-21 12:07:43,475 - WARNING - Add of existing embedding ID: 1003772251
2025-03-21 12:07:43,475 - WARNING - Insert of existing embedding ID: 1003772251
2025-03-21 12:07:43,477 - WARNING - Add of ex

Stored 1538 documents in ChromaDB.


In [20]:
import google.generativeai as genai
import os
from dotenv import load_dotenv

# Load variables from .env file
load_dotenv()

GEMINI_API_KEY = os.getenv("GEMINI_API_KEY")
genai.configure(api_key=GEMINI_API_KEY)

In [21]:
def rag_query(question):
    model = SentenceTransformer(EMBEDDING_MODEL)
    query_embedding = model.encode([question], convert_to_tensor=True).tolist()

    results = collection.query(
        query_embeddings=query_embedding,
        n_results=3  
    )
    context = "\n\n".join(['\n'.join([res['name'],res["description"]]) for res in results["metadatas"][0]])
    print(f'=== context === \n {context}')
    prompt = f"Using the following context:\n{context}\nAnswer the question: {question}"
    response = genai.GenerativeModel("gemini-2.0-flash").generate_content(prompt)

    return response.text

Test query

In [23]:
answer = rag_query("nước hoa cho nam mùi hương nhẹ nhàng")
print("Generated Response:", answer)

2025-03-21 12:09:11,121 - INFO - Use pytorch device_name: mps
2025-03-21 12:09:11,121 - INFO - Load pretrained SentenceTransformer: paraphrase-multilingual-mpnet-base-v2
Batches: 100%|██████████| 1/1 [00:00<00:00, 11.72it/s]


=== context === 
 Versace Pour Homme Dylan Blue
Hương đầu: Hương nước, Cam Bergamot Calabria, Quả bưởi, Lá sung Hương giữa: Lá hoa tím, Cây hoắc hương, Tiêu đen, Hương Ambroxan, Giấy cói Hương cuối: Xạ hương, Nhang (Hương), Đậu Tonka, Saffron Nếu như có ai hỏi tôi đã bao giờ bị một ai đó mê hoặc bởi mùi hương hay chưa, có lẽ tôi sẽ không do dự mà nói rằng đã từng, và trong đầu nghĩ ngay đến mùi hương man mát, nam tính được toả ra bởi Versace Pour Homme Dylan Blue. Tôi bị ấn tượng bởi những nốt hương của chai nước hoa này bởi sự mở đầu tinh khiết, trong trẻo của hương nước, Cam Bergamot và Bưởi. Nhưng trái với sự phóng khoáng, tươi mới ấy, Dylan Blue còn hấp dẫn tôi bởi cái sự nam tính của Hương Ambroxan, sự gai góc đầy cá tính của Tiêu đen. Chàng ta còn có thêm cái ấm áp, ngọt ngào của Xạ hương và Đậu tonka. Thứ hương đầy gợi cảm ấy như một cái bẫy ngọt ngào khiến bao con mồi tự sa vào. Bạn biết đó, tôi cũng là một trong số những con mồi bị thu hút bởi thứ hương sạch sẽ, cuốn hút ấy.

